In [1]:
#import nba_api as nb
import numpy as np
import pandas as pd
import scipy as sp
import math

In [1523]:
from nba_api.stats.endpoints import playercareerstats as pcs
from nba_api.stats.endpoints import boxscoreadvancedv2 as adv
# Anthony Davis
#career = pcs.PlayerCareerStats(player_ID="203076")
#career.get_data_frames()[0]

In [1524]:
from nba_api.stats.static import players

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
#print("Number of players fetched: {}".format(len(nba_players)))
#nba_players[:5]

In [6]:
players = []
i = 0
for player in nba_players:
    ID = player["ID"]
    career = pcs.PlayerCareerStats(player_id=ID)
    players.append(career.get_data_frames()[0])
    i = i + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [12]:
#Makes a list of seasons by year

years = []
season = "1983"+"-84"
#years.append('Name')
while int(season[:4]) < 2024:
    years.append(season)
    year1 = int(season[:4])+1
    year2 = int(season[5:7])+1
    if year2 == 100:
        year2 = 0
        season = str(str(year1) + '-' + str(year2) + str(year2))
    if year2 < 10:
        season = str(str(year1) + '-' + '0' + str(year2))
    elif year2 != 100:
        season = str(str(year1) + str(-year2))

In [13]:
#Makes a DF of points/game each year for each player

temp = 0
ppg_df = pd.DataFrame(columns=years)
j = 0
ID = []
for player in players:
    i = 0
    for year in player["SEASON_ID"]:
        if int(year[0:4]) == 2024:
            p = 1
        elif int(year[0:4]) >= 1983:
            if temp == year:
                temp = year
                ppg_df.at[j, temp] = [player["PTS"][i]/player["GP"][i],player["PLAYER_ID"][0]]
                i = i + 1     
            elif temp != year:
                temp = year
                ppg_df.at[j, year] = [player["PTS"][i]/player["GP"][i],player["PLAYER_id"][0]]
                i = i + 1
        elif int(year[0:4]) < 1983:
            if temp == year:
                temp = year
                i = i + 1
            elif temp != year:
                temp = year
                i = i + 1
    if int(temp[0:4]) >= 1983:
        j = j + 1 
    if not player.empty:
        ID.append(player["PLAYER_ID"][0])

In [15]:
#Applies each players PPG to a year
columns = list(ppg_df)

,1983-84,1984-85,1985-86,1986-87,1987-88,1988-89,1989-90,1990-91,1991-92,1992-93,...,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[3.13953488372093, 76001]","[6.084507042253521, 76001]","[7.706666666666667, 76001]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[21.4625, 76003]","[21.962025316455698, 76003]","[23.367088607594937, 76003]","[17.512820512820515, 76003]","[14.5625, 76003]","[10.108108108108109, 76003]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[14.059701492537313, 51]","[10.333333333333334, 51]","[19.17283950617284, 51]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[1.2105263157894737, 1627757]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[5.454545454545454, 1627835]","[4.037037037037037, 1627835]",NaN,NaN,NaN,NaN,NaN,NaN
3450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"[3.71875, 1627790]","[7.779661016949152, 1627790]","[4.363636363636363, 1627790]",NaN,NaN,NaN,NaN


In [26]:
#Finds and stores the index of top 40 scorers that season

avg_pts_index = []
for column in columns:
    i = 0
    if column == 'Name':
        i = 1
    elif column != 'Name':
        a = ppg_df.sort_values(by=[column], ascending=False)
        temp1 = []
        for values in a[column]:
            i = i + 1
            temp1.append(values[1])
            if i == 40:
                break
            #row_number = a.index[a[column] == values]
            #temp.append(row_number)
            #print(temp1)
        avg_pts_index.append(temp1)

In [30]:
#Restructures indecis of highest averages to make more sense

i = 0
avg = []
while i < len(avg_pts_index):
    temp = []
    j = 0
    while j < 40:
        temp.append(avg_pts_index[i][j])
        j = j + 1
    i = i + 1
    avg.append(temp)

In [18]:
#Stores all player data

# open file
with open('nba.txt', 'w+') as f:
    
    # write elements of list
    for items in players:
        f.write('%s\n' %items)
    
    print("File written successfully")


# close the file
f.close()

File written successfully


In [19]:
#Points per Possession for each Season

ppp = [1.8,1.8,1.8,1.9,1.8,1.8,1.9,1.8,1.9,1.9,1.7,1.9,1.8,1.7,1.6,1.3,1.5,1.4,1.5,1.4,1.4,1.7,1.7,1.7,1.8,1.9,1.8,1.8,1.5,1.7,1.7,1.6,1.7,1.9,1.9,1.11,1.11,1.13,1.13,1.16,1.16]

41

In [20]:
#Finds, reads, and neatly stores all necessary player data into a DF, accounting for several errors

temp = 0
df = pd.DataFrame(columns=years)
j = 0
for player in players:
    i = 0
    temp2 = '0000'
    temp1 = '0000'
    for year in player["SEASON_ID"]:
        if int(year[0:4]) >= 1983:
            if int(year[0:4]) == 2024:
                a = 0
            elif temp2 == year:
                temp2 = year
                minutes = player["MIN"][i]
                two_point_fieldgoals = (player["FGA"][i] - player["FG3A"][i])/minutes
                two_point_percentage = (player["FGM"][i]-player["FG3M"][i])/minutes/two_point_fieldgoals
                offensive_rebounds = (player["OREB"][i])/minutes
                defensive_rebounds = (player["DREB"][i])/minutes
                free_throws = (player["FTA"][i])/minutes
                free_throw_percentage = (player["FT_PCT"][i])
                three_point_fieldgoals = (player["FG3A"][i])/minutes
                three_point_percentage = (player["FG3_PCT"][i])
                blocks = (player["BLK"][i])/minutes
                steals = (player["STL"][i])/minutes
                turnovers = (player["TOV"][i])/minutes
                assists = (player["AST"][i])/minutes
                personal_fouls = (player["PF"][i])/minutes
                player_id = player["PLAYER_ID"][i]
                temp1 = year
                df.at[j, temp2] = [two_point_fieldgoals,two_point_percentage,offensive_rebounds,defensive_rebounds,free_throws,free_throw_percentage,three_point_fieldgoals,three_point_percentage,blocks,steals,turnovers,assists,personal_fouls,player_id,minutes]
                i = i + 1
            elif temp1 == year:
                temp1 = year
                minutes = player["MIN"][i]
                two_point_fieldgoals = (player["FGA"][i] - player["FG3A"][i])/minutes
                two_point_percentage = (player["FGM"][i]-player["FG3M"][i])/minutes/two_point_fieldgoals
                offensive_rebounds = (player["OREB"][i])/minutes
                defensive_rebounds = (player["DREB"][i])/minutes
                free_throws = (player["FTA"][i])/minutes
                free_throw_percentage = (player["FT_PCT"][i])
                three_point_fieldgoals = (player["FG3A"][i])/minutes
                three_point_percentage = (player["FG3_PCT"][i])
                blocks = (player["BLK"][i])/minutes
                steals = (player["STL"][i])/minutes
                turnovers = (player["TOV"][i])/minutes
                assists = (player["AST"][i])/minutes
                personal_fouls = (player["PF"][i])/minutes
                player_id = player['PLAYER_ID'][i]
                temp1 = year
                temp2 = temp1
                df.at[j, temp1] = [two_point_fieldgoals,two_point_percentage,offensive_rebounds,defensive_rebounds,free_throws,free_throw_percentage,three_point_fieldgoals,three_point_percentage,blocks,steals,turnovers,assists,personal_fouls,player_id,minutes]
                i = i + 1     
            elif temp1 != year:
                minutes = player["MIN"][i]
                TwoFG = (player["FGA"][i] - player["FG3A"][i])/minutes
                two_point_percentage = (player["FGM"][i]-player["FG3M"][i])/minutes/two_point_fieldgoals
                offensive_rebounds = (player["OREB"][i])/minutes
                defensive_rebounds = (player["DREB"][i])/minutes
                free_throws = (player["FTA"][i])/minutes
                free_throw_percentage = (player["FT_PCT"][i])
                three_point_fieldgoals = (player["FG3A"][i])/minutes
                three_point_percentage = (player["FG3_PCT"][i])
                blocks = (player["BLK"][i])/minutes
                steals = (player["STL"][i])/minutes
                turnovers = (player["TOV"][i])/minutes
                assists = (player["AST"][i])/minutes
                personal_fouls = (player["PF"][i])/minutes
                player_id = player['PLAYER_ID'][i]
                temp1 = year
                df.at[j, temp1] = [two_point_fieldgoals,two_point_percentage,offensive_rebounds,defensive_rebounds,free_throws,free_throw_percentage,three_point_fieldgoals,three_point_percentage,blocks,steals,turnovers,assists,personal_fouls,player_id,minutes]
                i = i + 1
        elif int(year[0:4]) < 1983:
            if int(year[0:4]) == 2024:
                a = 0
            elif temp1 == year:
                temp1 = year
                i = i + 1
            elif temp1 != year:
                temp1 = year
                i = i + 1
    if int(temp1[0:4]) >= 1983:
        j = j + 1

<ipython-input-20-b3f4e0993da3>:56: RuntimeWarning: invalid value encountered in double_scalars
  TwoPct = (player["FGM"][i]-player["FG3M"][i])/MIN/TwoFG
<ipython-input-20-b3f4e0993da3>:36: RuntimeWarning: invalid value encountered in double_scalars
  TwoPct = (player["FGM"][i]-player["FG3M"][i])/MIN/TwoFG
<ipython-input-20-b3f4e0993da3>:55: RuntimeWarning: invalid value encountered in true_divide
  TwoFG = (player["FGA"][i] - player["FG3A"][i])/MIN
<ipython-input-20-b3f4e0993da3>:56: RuntimeWarning: invalid value encountered in true_divide
  TwoPct = (player["FGM"][i]-player["FG3M"][i])/MIN/TwoFG
<ipython-input-20-b3f4e0993da3>:57: RuntimeWarning: invalid value encountered in true_divide
  OREB = (player["OREB"][i])/MIN
<ipython-input-20-b3f4e0993da3>:58: RuntimeWarning: invalid value encountered in true_divide
  DREB = (player["DREB"][i])/MIN
<ipython-input-20-b3f4e0993da3>:59: RuntimeWarning: invalid value encountered in true_divide
  FT = (player["FTA"][i])/MIN
<ipython-input-20-b3

In [38]:
#Finds Averages per seassaon of a few important stats

columns = list(df)
avg = []
metric_adjustment = []
k = 0

for column in columns:
    temp = []
    j = 0
    two_point_sum = 0
    three_point_sum = 0
    free_throw_sum = 0
    personal_foul_sum = 0
    [two_point_fieldgoals,two_point_percentage,offensive_rebounds,defensive_rebounds,free_throws,free_throw_percentage,three_point_fieldgoals,three_point_percentage,blocks,steals,turnovers,assists,personal_fouls,player_id]
    while j < 15:
        i = 0
        s = 0
        n = 0
        while i < 1876:
            #print(i)
            if type(df[column][i]) == list:
                if df[column][i][j] is None:
                    if (df[column][i][13] in avg_pts_index[k]) == False:
                        #print(type(df[column][i][j]))
                        #print(i)
                        #print(j)
                        n = n + 1
                elif df[column][i][j] is not None:
                    if math.isnan(df[column][i][j]) == True:
                        p = 0
                    elif math.isnan(df[column][i][j]) == False:
                        if df[column][i][14] > 100:
                            if (df[column][i][13] in avg_pts_index[k]) == False:
                            #print(type(df[column][i][j]))
                                if j == 0:
                                    two_point_sum = two_point_sum + df[column][i][j]
                                if j == 6:
                                    three_point_sum = three_point_sum + df[column][i][j]
                                if j == 4:
                                    free_throw_sum = free_throw_sum + df[column][i][j]
                                if j == 12:
                                    personal_foul_sum = personal_foul_sum + df[column][i][j]                               
                                n = n + 1
                                s = s + df[column][i][j]
            i = i + 1
        temp.append(s/n)
        j = j + 1
    avg.append(temp)
    metric_adjustment.append([two_point_sum*2/(three_point_sum*3+two_point_sum*2)*2 + three_point_sum*3/(three_point_sum*3+two_point_sum*2)*3,free_throw_sum/personal_foul_sum*avg[k][5],ppp[k]])
    k = k + 1
    

In [39]:
# Calculates the metric for all players with more than 100 minutes

i = 0
player_performance_metric = []
for year in df:
    for player in df[year]:
        temp = 0
        if type(player) != list:
            a = 1;
        if type(player) == list:
            if player[14] > 100:
                two_point_fieldgoals = player[0]/avg[i][0]
                two_point_percentage = player[1]/avg[i][1]
                offensive_rebounds = player[2]/avg[i][2]
                defensive_rebounds = player[3]/avg[i][3]
                free_throws = player[4]/avg[i][4]
                if player[5] is not None:
                    free_throw_percentage = player[5]/avg[i][5]
                if player[5] is None:
                    free_throw_percentage = 0
                three_point_fieldgoals = player[6]/avg[i][6]
                if player[7] is not None:
                    three_point_percentage = player[7]/avg[i][7]
                    if math.isnan(three_point_percentage):
                        three_point_percentage = 0
                if player[7] is None:
                    three_point_percentage = 0
                blocks = player[8]/avg[i][8]
                steals = player[9]/avg[i][9]
                turnovers = player[10]/avg[i][10]
                assists = player[11]/avg[i][11]
                personal_fouls = player[12]/avg[i][12]
                player_id = player[13]
                MET = ((TwoFG*two_point_percentage+blocks/2)*2+three_point_fieldgoals*three_point_percentage*3+(FT*free_throw_percentage-PF*(metric_adjustment[i][1])))+assists*0.6*(metric_adjustment[i][0])
                +(steals-turnovers+offensive_rebounds+defensive_rebounds/2)*(metric_adjustment[i][2])
                if math.isnan(MET) == True:
                    print(three_point_percentage)
                k = 0
                for player_ids in player_performance_metric:
                    if player_id == player_ids[0][1]:
                        temp = 1
                        break;
                    k = k + 1     
                if temp == 0:
                    player_performance_metric.append([[MET,player_id]])
                elif temp == 1:
                    player_performance_metric[k].append([MET,player_id])
    i = i + 1

1.5835084700339934


In [42]:
#Creates a list of player IDs and reformats the metric to not include ID

i = 0
ID = []
for player in player_performance_metric:
    j = 0
    ID.append(str(player_performance_metric[i][j][1]))
    for year in player: 
        player_performance_metric[i][j] = player_performance_metric[i][j][0]
        j = j + 1
    i = i + 1

In [318]:
# Pads the metric for processing
max_length = max([np.array(player_performance_metric[i]).shape[0] for i in range(len(player_performance_metric))])

# Pad the time series to the same length
X_padded = []
for i in range(len(player_performance_metric)):
    padded = np.pad(player_performance_metric[i], ((0, max_length - np.array(player_performance_metric[i]).shape[0])), 'constant')
    X_padded.append(padded)
X_padded = np.array(X_padded)
player_performance_metric = X_padded.tolist()

In [ ]:
# Saves the padded metric as a dataframe
df2 = pd.DataFrame(np.array(player_performance_metric).transpose(),
                   columns=ID)
df2.to_csv('player_performance_metric.csv', index=False)

In [3]:
# Imports metric dataframe if already calculated
import pandas as pd
import numpy as np
df = pd.read_csv('player_performance_metric.csv')
player_performance_metric = []
for i in df:
    player_performance_metric.append(df[i].values)

In [5]:
# Adds a different metric list that includes the year each player has been in the league
player_performance_metric = []
player_performance_year = []
for player in df:
    temp = []
    temp2 = []
    j = 0
    for year in df[player]:
        temp.append(year)
        temp2.append(j)
        j = j + 1
    player_performance_metric.append(temp)
    player_performance_year.append(temp2)
player_performance_metric = np.array(player_performance_metric).reshape(2818,22,1)

In [7]:
# Cleans up the metric slightly

j = 0
player_performance_metric_adjusted = []
for player in player_performance_metric:
    i = 0
    temp = []
    for year in player:
        #print(player_performance_metric[j][i][0])
        temp.append([player_performance_metric[j][i][0],player_performance_year[j][i]+1])
        i = i + 1
    player_performance_metric_adjusted.append(temp)
    j = j + 1

In [8]:
#changes year 22 to be 0 to make scaling not go to 0
#the 1 datapoint of a player playing in year 22 is not that important

i = 0
for player in player_performance_metric_adjusted:
    j = 0
    for year in player:
        if year[1] == 22:
            player_performance_metric_adjusted[i][j][1] = 0
        j = j + 1
    i = i + 1

In [11]:
#cleaning up and saling

player_performance_scaled = []
i = 0
while i < len(player_performance_metric_adjusted):
    player_performance_scaled.append(player_performance_metric_adjusted[i][0:22])
    i = i + 1
#Scaling data and accounting for error
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
player_performance_scaled = np.array(player_performance_scaled).reshape(-1,2).tolist()
scaler = scaler.fit(player_performance_scaled)
normalized = scaler.transform(player_performance_scaled)
player_performance_scaled = normalized

In [769]:
#Separating into X (input) and Y (target) data
#Here Y is set to be a year after X so that I can predict the next year based on the current year
X = []
Y = []
i = 0
while i < len(player_performance_scaled):
    X.append(player_performance_scaled[i:i+20])
    Y.append(player_performance_scaled[i+1:i+21])
    i = i + 22

In [715]:
#Scaling data and accounting for error
X = np.array(X).reshape(-1,2).tolist()
Y = np.array(Y).reshape(-1,2).tolist()

In [716]:
#The last year in a players' career currently teaches the model that 
#eventually the metric decays to 0
#I want growth
i = 0
while i < len(X):
    if X[i][0] != 0 and Y[i][0] == 0:
        X[i][0] = 0
    i = i + 1

In [717]:
np.shape(Y)

(56360, 2)

In [718]:
i = 0
for player in Y:
    j = 0
    for year in player:
        if math.isnan(year) == True:
            Y[i][j] = 0
        j = j + 1
    i = i + 1
i = 0
for player in X:
    j = 0
    for year in player:
        if math.isnan(year) == True:
            X[i][j] = 0
        j = j + 1
    i = i + 1

Deep Learning Implementation

When I started this project, I started it with deep learning in mind. After nearly completing this portion, I learned that this type of problem is more commonly solved with standard regressions. This portion covers my deep learning approach. I believe this appraoch still works, but as you can see later, a simple regression is able to come to very similar conclusions (according to R^2) with much less time and resources required. 

In [22]:
#Split data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,shuffle=False)
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [24]:
# Prepare the model
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras import regularizers

In [92]:
#Alternate model preparation
tf.keras.utils.set_random_seed(2)
model = keras.Sequential()
model.add(keras.layers.Input(shape=(2, 1)))
model.add(keras.layers.Masking(mask_value=0.))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(64,kernel_regularizer=regularizers.l1(0.000001),activation='relu'))
model.add(keras.layers.Dense(2,kernel_regularizer=regularizers.l1(0.000001),activation='sigmoid'))
#model.add(keras.layers.Dropout(0.4))
#model.add(keras.layers.Dense(64, activation="relu"))
#model.add(keras.layers.Dropout(0.4))
#model.add(keras.layers.Dense(5, activation="softmax"))

In [94]:
# Run the NN

my_adam_optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=my_adam_optimizer, loss='mean_absolute_error', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=100)

Epoch 1/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9611 - loss: 0.0660
Epoch 2/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9834 - loss: 0.0084
Epoch 3/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9835 - loss: 0.0078
Epoch 4/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9835 - loss: 0.0068
Epoch 5/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9838 - loss: 0.0065
Epoch 6/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9842 - loss: 0.0075
Epoch 7/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9838 - loss: 0.0062
Epoch 8/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9831 - loss: 0.0063
Epoch 9/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9841 - loss: 0.0060
Epoch 10/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9840 - loss: 0.0059
Epoch 11/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9837 - loss: 0.0057
Epoch 12/20
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

In [96]:
# Check results
results = model.evaluate(X_test, Y_test, batch_size=20)

846/846 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9882 - loss: 0.0029


In [98]:
predicts = model.predict(X_test)

529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [100]:
r2_data = []
r2_test = []
j = 0
for i in predicts:
    r2_data.append(i[0])
    r2_test.append(Y_test[j][0])
    j = j + 1

In [102]:
import sklearn
sklearn.metrics.r2_score(r2_test, r2_data)

0.9618328272648269

In [50]:
Y_test[31]

array([0.05193367, 0.23809524])

In [90]:
model.predict(np.array([X_test[31]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[[2.6950022e-09, 3.3211312e-01],
        [2.6950022e-09, 3.3211312e-01]]], dtype=float32)

Decision Tree Regression

I know ahead of time that each player's data will have vastly different shapes, so I linear/polynomial/exponential regressions won't do much good.
I could do something more complicated like giving each player different order polynomials, but I believe a decision tree regression is the most simple solution

Implementation of the decision tree is extremely simple - the one issue is that overfitting will almost certainly happen by default. Unlike previously, the decision tree overfitting on the training data made it unable to recognize the fitting data. To remedy this, minimum samples and maximum depth can be tunes

We do have to process the data differently - due to dimensional limits of the regressor, I can no longer include the year the player has been in the league, data that was crucial for deep learning. Furthermore, a good chunk of the R^2 diminishes since the regressor will predict a value other than 0 for the players most recently played season. To accurately show R^2, this is fixed before the calculation. I would un-pad the data beforehand, but to keep context for the regressor from going inbetween players, I have to keep those arrays separated in model fitting.

In [106]:
from sklearn.tree import DecisionTreeRegressor

In [861]:
x = []
y = []
i = 0
while i < len(X):
    j = 0
    temp = []
    temp2 = []
    while j < 20:
        temp.append(X[i][j][0])
        temp2.append(Y[i][j][0])
        j = j + 1
    i = i + 1
    x.append(temp)
    y.append(temp2)

In [863]:
i = 0
for player in y:
    j = 0
    for year in player:
        if math.isnan(year) == True:
            y[i][j] = 0
        j = j + 1
    i = i + 1
i = 0
for player in x:
    j = 0
    for year in player:
        if math.isnan(year) == True:
            x[i][j] = 0
        j = j + 1
    i = i + 1

In [865]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3,shuffle=False)

In [1142]:
regressor = DecisionTreeRegressor(random_state = 0, min_samples_leaf=2, max_depth=9)  
  
# fit the regressor with X and Y data 
regressor.fit(X_train, Y_train) 

DecisionTreeRegressor(max_depth=9, min_samples_leaf=2, random_state=0)

In [1143]:
predicts_train = regressor.predict(X_train)
predicts_test = regressor.predict(X_test)

In [1144]:
r2_data_train = []
r2_test_train = []
k = 0
for i in predicts_train:
    l = 0
    for j in i:
        r2_data_train.append(j)
        r2_test_train.append(Y_train[k][l])
        l = l + 1
    k = k + 1

In [1145]:
r2_data_test = []
r2_test_test = []
k = 0
for i in predicts_test :
    l = 0
    for j in i:
        r2_data_test.append(j)
        r2_test_test.append(Y_test[k][l])
        l = l + 1
    k = k + 1

In [1147]:
i = 0
while i < len(r2_test_test):
    if r2_data_test[i] != 0 and r2_test_test[i] == 0:
       r2_data_test[i] = 0
    i = i + 1
i = 0
while i < len(r2_test_train):
    if r2_data_train[i] != 0 and r2_test_train[i] == 0:
       r2_data_train[i] = 0
    i = i + 1
print(sklearn.metrics.r2_score(r2_test_test, r2_data_test))
print(sklearn.metrics.r2_score(r2_test_train, r2_data_train))

0.9515918045811398
0.9527937969356284
